In [34]:
import pickle
import pandas as pd

from data.make_dataset import load_data
from features.build_features import multi_ordinal_encoder, get_best_correlation_columns, choose_columns

In [35]:
decision_tree_model = pickle.load(open('models/models/decision_tree_regressor_model.sav', 'rb'))
random_forest_model = pickle.load(open('models/models/random_forest_regressor_model.sav', 'rb'))
ada_boost_model = pickle.load(open('models/models/ada_boost_regressor_model.sav', 'rb'))
xgboost_model = pickle.load(open('models/models/xgboost_regressor_model.sav', 'rb'))

c:\Users\MatiasMaiorano\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\MatiasMaiorano\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\MatiasMaiorano\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator AdaBoostRegressor from version 1.0.2 when using version 1.1.1. This might lead to breaking 

In [36]:
from sklearn.preprocessing import OrdinalEncoder

In [37]:
df = load_data('../data/raw/train.csv')
oe = OrdinalEncoder()
df = multi_ordinal_encoder(df, oe)

best_columns = get_best_correlation_columns(df, 'SalePrice', .5)
X_test = choose_columns(df, best_columns)
y_test = choose_columns(df, ['SalePrice'])

In [38]:
predict_decision_tree = decision_tree_model.predict(X_test)
predict_random_forest = random_forest_model.predict(X_test)
predict_ada_boost = ada_boost_model.predict(X_test)
predict_xgboost = xgboost_model.predict(X_test)

In [63]:
df_ris = pd.DataFrame(columns=['y_true'])
df_ris.y_true = y_test.values.squeeze()
df_ris['predict_decision_tree'] = predict_decision_tree
df_ris['predict_random_forest'] = predict_random_forest
df_ris['predict_ada_boost'] = predict_ada_boost
df_ris['predict_xgboost'] = predict_xgboost
df_ris

,y_true,predict_decision_tree,predict_random_forest,predict_ada_boost,predict_xgboost
0,208500,189500.0,195021.4675,202762.806167,207833.015625
1,181500,165000.0,168039.9225,157621.141026,179248.750000
2,223500,189500.0,211245.4675,207429.728682,220799.234375
3,140000,111250.0,173976.2175,179754.486811,139703.328125
4,250000,262000.0,276499.4750,269375.195980,250897.546875
...,...,...,...,...,...
1455,175000,175950.0,178194.9950,173822.851562,173729.609375
1456,210000,239000.0,212841.0700,215953.126394,210353.343750
1457,266500,263750.0,265100.9825,230742.149780,266509.187500
1458,142125,140000.0,134817.7150,133844.104167,140874.437500
